In [1]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Optional
import pandas as pd
from pydantic import BaseModel


In [46]:
## Ingredientai
resp = requests.get("https://www.15min.lt/maistas/receptas/varskes-ir-fermentinio-surio-pyragas-8796")
soup = BeautifulSoup(resp.content, "html.parser")
all_ingredients: List[Dict] = []

recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
recipe_image = soup.find("div", class_="image").find("img").get("src")
recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
for ingredient in recipe_ingredients:
    the_ingredient = ingredient.find_all("span")
    all_ingredients.append({
        "ingredient_name": the_ingredient[0].text.strip(),
        "ingredient_amount": the_ingredient[2].text.strip(),
        "ingredient_note": the_ingredient[3].text.strip()
    })

#all_ingredients

## Paruosimo budas
recipe_manual: List[str] = []
recipe_making_steps = soup.find("div", class_="description text").find_all("p")

recipe_making_steps



for step in recipe_making_steps:
    stepsas = step.text.strip()
    recipe_manual.append(stepsas)

In [35]:
## Info apie viena recepta

def extract_recipe_ingredients(url: str) -> str:
    ## Ingredientai
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    all_ingredients: List[Dict] = []

    recipe_title = soup.find("div", class_="recipe-head").find("h1").text.strip()
    recipe_image = soup.find("div", class_="image").find("img").get("src")
    recipe_ingredients = soup.find("ul", class_="ingredients").find_all("li")
    for ingredient in recipe_ingredients:
        ingredient_span = ingredient.find_all("span")
        all_ingredients.append({            
                "ingredient_name": ingredient_span[0].text.strip(),
                "ingredient_amount": ingredient_span[2].text.strip(),
                "ingredient_note": ingredient_span[3].text.strip()
            })
       
    return str(all_ingredients)


def extract_making_steps(url: str) -> str:

    ## Paruosimo budas
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")
    recipe_manual: List[str] = []
    recipe_making_steps = soup.find("div", class_="description text").find_all("p")

    for step in recipe_making_steps:
        step_to_txt = step.text.strip()
        recipe_manual.append(step_to_txt)
    
    return "\n".join(recipe_manual) 


In [10]:
## IKEA search products

# sudeda paieskos rezultatos is pirmo psl

def search_products(url) -> List[Dict]:
    """Function for doing search in ikea.lt"""
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    products: List[Dict] = []

    search_results_all = soup.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
    for product in search_results_all:
        link = product.find("div", class_="itemInfo").a.get("href")
        products.append({            
                    "product_name": product.find("div", class_="itemInfo").a.text.strip(),
                    "product_info": product.find("div", class_="itemInfo").find("div", class_ = "itemPrice-wrapper").span.text.replace(" €", ""),
                    "product_link": f"https://www.ikea.lt{link}",
                    "product_image": product.find("div", class_ = "productImg").img.get("src")
                })

    return products
    
#prod = search_products("https://www.ikea.lt/lt/search/?q=kede")
#prod

In [61]:
## IKEA retrieve links

from pydantic import BaseModel

__items_per_page__: int = 40
__domain__: str = "https://www.ikea.lt"

class FurnitureLink(BaseModel):
    url: str


class Furniture(BaseModel):
    """Class for base properties of furniture"""
    furniture_name: str
    furniture_description: str
    furniture_price = int
    furniture_image_link: List[str]    
    furniture_key_features: str
    furniture_size: str
    
    furniture_stock_in_store: int


def _get_page_content(query: str) -> Optional[BeautifulSoup]:
        """Method to get needed content from search result page or whatever related page."""
        resp = requests.get(f"{__domain__}/{query}")
        if resp.status_code == 200:
            return BeautifulSoup(resp.content)
        raise Exception("Cannot get content. Site is unreachable.")



#################################################

def _retrieve_items_links(results_count: int, keyword: str) -> List[str]:  #[FurnitureLink]: # prideti self
    """Method to search furnitures by keyword and save specifed number of results."""
    results: List[str] = []  #[FurnitureLink] = []        
    pages_to_iterate: int = (results_count // __items_per_page__) + 1 # kiek paieskos rezultatu puslapiu naudoti        
    
    for page_num in range(1, pages_to_iterate + 1):
        content = _get_page_content(f"/lt/search/?q={keyword}&page={page_num}") #pries metoda prideti self
        pages_count = content.find("ul", class_="pagination mb-0")
        no_results = content.find("div", class_="col mt-4").h3.text        
        
        if pages_count != None:
            max_number_of_pages = int(content.find("ul", class_="pagination mb-0").find_all("li", class_="page-item")[3].text)
        elif no_results == "Rezultatų nerasta":
            max_number_of_pages = 0
        else:
            max_number_of_pages = 1
        
        if content:
            if max_number_of_pages >= pages_to_iterate:                        
                all_items_per_page = content.find_all("div", class_="col-6 col-md-4 col-lg-3 p-0 itemBlock")
                counter = 1

                while results_count >= counter:                                                                    
                    for item in all_items_per_page:
                        link = item.find("div", class_="itemInfo").a.get("href")
                        item = f"{__domain__}{link}"
                        results.append(item)  #(FurnitureLink(url = item))
                        counter += 1
            else: 
                print(f"Search has less results than requested.")
                break
        if not content:
            break

    return results
    

In [62]:
test = _retrieve_items_links(0, "bybis")
print(test)




Search has less results. Please enter new number.
[]


In [22]:
## IKEA get key features

def _get_key_features(content: BeautifulSoup) -> str:  # pridet self
    results: List[str] = []   
    features = content.find("div", class_="tab-pane_box").find_all("p")
    
    for feature in features: 
        feature_line = feature.text.strip()
        results.append(feature_line)       
        
    return results

In [20]:
## IKEA extract main info
resp = requests.get("https://www.ikea.lt/lt/products/valgomasis/sedimieji-valgomojo-baldai/kedes-ir-suoliukai/terje-sulankstomoji-kede-sviesi-pilka-balta-art-80456982")
#resp = requests.get("https://www.ikea.lt/lt/products/ikea-verslui/nekilnojamasis-turtas/sofos/jattebo-dviviete-moduline-sofa-tamsi-geltona-zalia-spr-29471405")
content = BeautifulSoup(resp.content, "html.parser")



##############################

def _extract_main_info(content: BeautifulSoup) -> str: # prideti self
        """Method to get ingredients of the recipe."""
        results: List[Dict] = []
        
        main_info = content.find("ul", class_="ingredients").find_all("li")
        for ingredient in recipe_ingredients:
            ingredient_span = ingredient.find_all("span")
            all_ingredients.append({
                "ingredient_name": ingredient_span[0].text.strip(),
                "ingredient_amount": ingredient_span[2].text.strip(),
                "ingredient_note": ingredient_span[3].text.strip()
            })
        return str(results)



def _retrieve_furniture_info(link: str) -> Optional[Furniture]: #prideti self, link: FurnitureLink prie attr
        """Method to get main info about furniture."""
        content = _get_page_content(link.url)  # prideti self prie get metodo

        if content:
            try:
                furniture_name = content.find("div", class_="d-flex align-items-center flex-wrap").h3.text
                furniture_description = content.find("h4", class_="itemFacts font-weight-normal").span.text
                furniture_price = int(content.find("div", class_="itemPrice-wrapper").p.span.text.replace(" €", ""))
            except AttributeError:
                return None

            try:
                furniture_image_link = content.find("a", class_ = "slideImg").find("img").get("src")
            except KeyError:
                furniture_image_link = None

            return Furniture(
                furniture_name = furniture_name,
                furniture_description = furniture_description,
                furniture_price = furniture_price,
                furniture_image_link = furniture_image_link,
                furniture_key_features = _get_key_features(content), # self
                furniture_size = str,

                furniture_stock_in_store = int,

                ingredients = self._extract_ingredients(content),
                making_steps = self._extract_making_steps(content)
            )
        else:
            return None

In [ ]:
## pabandyt su selenium, nes reikia hitint mygtuka

def _check_number_in_stock(content: BeautifulSoup) -> str: # self
    stock: str = content.find("div", class_="message noCursor d-flex").find("p").text
